In [ ]:
# default_exp core

# pytrim2

> A python program for trimming and demultiplexing naopore reads.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
# Import dependencies
from Bio import SeqIO
import numpy as np
from Bio import Align
from Bio import pairwise2

In [ ]:
#export

def findAdapter(seq_record, barcode_primer, cutOff):
    # Alignment using pairwise2
    alignments = pairwise2.align.localms(seq_record.seq, barcode_primer, 1, 0, -2, -2)

    # Find best match
    alingmentAccuracy = np.array( [i.score for i in alignments]  )
    alingmentEnd = np.array( [i.end for i in alignments]  )
    maxAcc = np.max(alingmentAccuracy)
    maxAcc = np.max(alingmentAccuracy)
    maxAccAlignment = np.where(alingmentAccuracy == maxAcc)[0]
    inwardAlingmentEnd = np.max(alingmentEnd[maxAccAlignment])
    
    maxAcc = maxAcc / len(barcode_primer) * 100

    if maxAcc >= cutOff:
        print("TRUE")
    maxAcc = maxAcc
    print("queried length:", len(seq_record.seq), "; accuracy [%]:", maxAcc)

In [ ]:
#export

def findAlingments(record_dict, barcode_primer, inward_end, max_alignments):
    
    record_keys = list(record_dict.keys())

    aligner = Align.PairwiseAligner()
    aligner.match_score = 1.0
    aligner.mismatch_score = 0
    aligner.gap_score = -2
    aligner.mode = "local"

    n_sequences = len(record_keys)

    array_cols = max_alignments + 2
    al_array = np.zeros( (n_sequences, array_cols) )

    for i in list(range(0, n_sequences, 1)):
        al = []
        seq = record_dict[record_keys[i]].seq[0:200]        
        alignments = aligner.align(seq, barcode_primer)
        len_alignments = len(alignments)
        if(len_alignments <= max_alignments):
            score = alignments.score
            al = [j.aligned for j in alignments]
            len_al = len(al)
            for k in range(0, len_al):
                al[k] = (al[k][0][0][1])
            al_array[i, 0:len(al)] = al
            al_array[i, -1] = len_al
            al_array[i, -2] = alignments.score/len(barcode_primer)
            
    return(al_array)
    
    

In [ ]:
#hide

record_dict = SeqIO.index("test_data/test.fasta", "fasta")
barcode_primer = "AGTGCCCCGCGCCACGCGCG"
ends = findAlingments(record_dict, barcode_primer, 200, 10)
print(ends)


[[ 20.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    1. ]
 [ 62.   73.    0.    0.    0.    0.    0.    0.    0.    0.    0.5   2. ]
 [192.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.5   1. ]]


In [ ]:
#slow
record_keys = list(record_dict.keys())
    
aligner = Align.PairwiseAligner()
aligner.match_score = 1.0
aligner.mismatch_score = 0
aligner.gap_score = -2
aligner.mode = "local"
   
max_alignments = 10
n_sequences = len(record_keys)

array_cols = max_alignments + 2
al_array = np.zeros( (n_sequences, array_cols) )

for i in list(range(0, n_sequences, 1)):
    al = []
    seq = record_dict[record_keys[i]].seq[0:200]        
    alignments = aligner.align(seq, barcode_primer)
    len_alignments = len(alignments)
    if(len_alignments <= max_alignments):
        score = alignments.score
        al = [j.aligned for j in alignments]
        len_al = len(al)
        for k in range(0, len_al):
            al[k] = (al[k][0][0][1])
        al_array[i, 0:len(al)] = al
        al_array[i, -1] = len_al
        al_array[i, -2] = alignments.score

